In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import median_filter
from scipy.io import loadmat
import sys
from pathlib import Path
import os
from typing import Dict
import collections
from mne_bids.path import get_bids_path_from_fname
import json

import mne
from mne.time_frequency import read_tfrs

mne.set_log_level("ERROR")
from mne_bids import BIDSPath, get_entities_from_fname, get_entity_vals, read_raw_bids
from mne.io import read_raw_edf

sys.path.append("..\episcalp")


In [12]:
# def load_raw_data(
#     raw_path, subjects=None, verbose=True, **kwargs) -> Dict:
#     """Read all patient's raw data files.

#     Parameters
#     ----------
#     raw_path : str | pathlib.Path
#         The path to the raw files. Inside this folder should
#         be a folder for each subject, stored in BIDS format.
#     subjects : [type], optional
#         Specific subjects to load, by default None, which loads all
#         subjects found in "raw_path"
#     verbose : bool, optional
#         verbosity, by default True
#     """
    
#     if subjects is None:
#         subjects = get_entity_vals(raw_path, "subject")

#     if verbose:
#         print(f"Loading data for subjects: {subjects} from {raw_path}")

#     # extract the BIDS root of the dataset
#     this_path = raw_path
#     while this_path.parent.name != "derivatives" and this_path.parent.name != "/":
#         this_path = this_path.parent
#     root = this_path.parent.parent

#     dataset = collections.defaultdict(list)
#     for subject in subjects:
#         subj_path = deriv_path / f"sub-{subject}"

#         # get all files of certain search_str
#         fpaths = subj_path.glob(search_str)

#         # now load in all file paths
#         for fpath in fpaths:
#             entities = get_entities_from_fname(fpath.name, on_error='ignore')
#             bids_path = BIDSPath(
#                 root=root, subject=entities['subject'],
#                 session=entities['session'],
#                 task=entities['task'],
#                 run=entities['run'],
#                 suffix='eeg',
#                 datatype='eeg',
#                 extension='.edf'
#             )

#             deriv = read_func(fpath, **kwargs)

#             # extract data
#             ch_names = deriv.ch_names
#             if isinstance(deriv, EpochsTFR):
#                 deriv_data = deriv.data
#                 deriv_data = _preprocess_epochs_tfr(deriv_data)
#             else:
#                 if "perturbmatrix" in fpath.name:
#                     deriv.normalize()
#                 deriv_data = deriv.get_data()

#             dataset["subject"].append(subject)
#             dataset["data"].append(deriv_data)
#             dataset["ch_names"].append(ch_names)
#             dataset["roots"].append(root)
#             dataset["bids_path"].append(bids_path)
#     return dataset

In [2]:
user = "patrick"
if user == "patrick":
    jhroot = Path("D:/OneDriveParent/OneDrive - Johns Hopkins/Shared Documents/bids")
    jeffroot = Path("D:/OneDriveParent/Johns Hopkins/Jefferson_Scalp - Documents/root")

    # not ready yet
    upmcroot = Path("/Users/adam2392/Johns Hopkins/UPMC_Scalp - Documents/")
    deriv_dir = Path(
        "D:/OneDriveParent/OneDrive - Johns Hopkins/Shared Documents/derivatives"
    )
elif user == "adam":
    jhroot = Path("/Users/adam2392/Johns Hopkins/Scalp EEG JHH - Documents/bids/")
    jeffroot = Path("/Users/adam2392/Johns Hopkins/Jefferson_Scalp - Documents/root/")

    # not ready yet
    upmcroot = Path("/Users/adam2392/Johns Hopkins/UPMC_Scalp - Documents/")
    deriv_dir = Path(
        "/Users/adam2392/Johns Hopkins/Scalp EEG JHH - Documents/derivatives"
    )
elif user == "kristin":
    jhroot = Path("D:/kgunnar1/OneDrive - Johns Hopkins/Documents - Scalp EEG JHH/bids")
#     jeffroot = Path("/Users/Kristin/OneDrive - Johns Hopkins/Documents - Jefferson_Scalp/root/")
    jeffroot = Path("D:/kgunnar1/OneDrive - Johns Hopkins/Documents - Jefferson_Scalp/root")  
    upmcroot = Path("D:/kgunnar1/OneDrive - Johns Hopkins/Documents - UPMC_Scalp/scalp_study/root")

    # not ready yet
#     upmcroot = Path("/Users/Kristin/OneDrive - Johns Hopkins/Documents - UPMC_Scalp")
#     deriv_dir = Path("/Users/Kristin/OneDrive - Johns Hopkins/Documents - Scalp EEG JHH/derivatives")
    deriv_dir = Path("D:/kgunnar1/OneDrive - Johns Hopkins/Documents - Scalp EEG JHH/derivatives")

In [3]:
def iis_detector(root, thresh,min_len,max_len, subjects=None, search_str="*.edf", verbose=True):
    
#     Function to computes spike rate per channel (spikes/sec)
#     Parameters
#     ----------
#     root : The path to the raw files. Inside this folder should
#         be a folder for each subject, stored in BIDS format.
#     thresh: Threshold for spike detector. #spikes are negative. Lower thresh = less spikes.
#     min_len: Minimum spike duration, in mseconds
#     max_len: Maximum spike duration, in mseconds
#     subjects : [type], optional
#         Specific subjects to load, by default None, which loads all
#         subjects found in "raw_path"
#     search_str: The file type storing the EEG data

#     verbose : bool, optional
#         verbosity, by default True

    '''
    # For testing (Anil's values):
    thresh = -.7 #spikes are negative. lower thresh = less spikes.
    min_len = 0.02  # was 0.2
    max_len = 0.2  # was 0.7 
    search_str="*.edf"
    '''

    """Read in raw data."""
    if subjects is None:
        subjects = get_entity_vals(root, "subject")

    if verbose:
        print(f"Loading data for subjects: {subjects}")

    dataset = collections.defaultdict(list)
    for subject in subjects:
        subj_path = root / f"sub-{subject}"

        # get all files of certain search_str
        fpaths = subj_path.rglob(search_str)

        # now load in all file paths and run spike analysis
        for fpath in fpaths:
            bids_path = get_bids_path_from_fname(fpath)
            edf_raw = read_raw_edf(bids_path)
            raw_data, times = edf_raw[:, :]
            fs = edf_raw.info["sfreq"]
            T = len(edf_raw)/fs;
            minspikeLen = round(min_len * fs) # 20 ms to 70 ms is the range Anil used
            maxspikeLen = round(max_len * fs)

            # filter signal
            # nchannels * time points
            processed = np.zeros(raw_data.shape)
            spikes = np.zeros(raw_data.shape)
            for channel in np.arange(raw_data.shape[0]):
                sig = raw_data[channel, :]
                sig = median_filter(sig, minspikeLen)
                sig = sig - median_filter(sig, maxspikeLen)

                sig = (sig - np.percentile(sig, 1))/(np.percentile(sig, 99) - np.percentile(sig, 1))
                processed[channel, :] = sig

            spikes = processed < thresh #manually chosen threshold
    #         print(np.sum(spikes))  

            #enforce 1 spike per region
            for chan in np.arange(spikes.shape[0]):
                spikeSig = spikes[chan, :]
                only = False
                for time in np.arange(spikes.shape[1]):
                    if spikeSig[time] == 1 and only:
                        spikes[chan, time] = 0
                    elif spikeSig[time] == 0:
                        only = False
                    else:
                        only = True
    #         print(np.sum(spikes)) #number of unique spikes

            # Get spike rate per channel
            ch_names = edf_raw.ch_names
            #pike_rates = np.zeros(raw_data.shape[0])
            spike_rate_dict = {}
            for chan, ch_name in enumerate(ch_names):
                spikeLocs = np.where(spikes[chan, :] == 1)[0]
                #pike_rates[chan] = len(spikeLocs)/T  # spikes/second
                spike_rate_dict[ch_name] = len(spikeLocs)/T  # spikes/second
    #         print(spike_rates)

            '''
            #Can plot spikes to see if they look okay
            for chan in np.arange(spikes.shape[0]):
                chan_name = edf_raw.ch_names[chan]
                sig = raw_data[chan, :]
                spikeLocs = np.where(spikes[chan, :] == 1)[0]
                for spk in spikeLocs:
                    plt.figure()
                    plt.plot(sig[(spk-100):(spk+100)])
                    # plt.title("Chan: " + str(chan) + " time: " + str(spk))
                    plt.title("Chan: "+ chan_name + " time: " + str(spk))
            '''
            # Save spike rates per channel
            # spike_root = deriv_root / "spikes_anil" / f"min-{minspikeLen}" / f"max-{maxspikeLen}" / f"thresh-{thresh}"
            spike_root = deriv_root / "spikes_anil" / f"thresh-{thresh}"
            subj_root = spike_root / f"sub-{subject}"
            subj_root.mkdir(parents=True, exist_ok=True)

            out_fpath = str(subj_root / fpath.with_suffix(".json").name) 
            with open(out_fpath, 'w+') as fid:
                json.dump(spike_rate_dict, fid, indent=4)
                
    #     return spike_rate_dict       


# Compute spike rates for different threshold values 

In [4]:
# test_thresh = np.arange(-1.3,-0.2,0.1)
# test_thresh = np.round(test_thresh,1)

# for thresh in test_thresh:
#     print(thresh)
    
    
# print(subject)

In [5]:
# Define directory of each raw datafile
bids_roots = [jhroot, jeffroot, upmcroot]

# Define spike criteria (Spikes: 20-70 msec, Sharp Waves: 70-200 msec)
min_len = 0.02  # (20 msec)
max_len = 0.2  # (200 msec)
# thresh values to test: [-1.5:0.1:-0.3]
test_thresh = np.arange(-1.3,-0.2,0.1)
test_thresh = np.round(test_thresh,1)

# datasets = []
# kwargs = {"preload": "True"}
# for root in bids_roots:
#     print(f")

for root in bids_roots:
    deriv_root = root / "derivatives"

    for thresh in test_thresh:
        iis_detector(root, thresh,min_len,max_len, subjects=None, search_str="*.edf", verbose=True)

Loading data for subjects: ['jhh001', 'jhh002', 'jhh003', 'jhh004', 'jhh005', 'jhh006', 'jhh007', 'jhh008', 'jhh009', 'jhh010', 'jhh011', 'jhh012', 'jhh013', 'jhh014', 'jhh015', 'jhh016', 'jhh017', 'jhh018', 'jhh019', 'jhh020', 'jhh021', 'jhh022', 'jhh023', 'jhh024', 'jhh025', 'jhh026', 'jhh027', 'jhh028', 'jhh029', 'jhh030', 'jhh031', 'jhh032', 'jhh033', 'jhh034', 'jhh035', 'jhh036', 'jhh037', 'jhh038', 'jhh101', 'jhh102', 'jhh103', 'jhh104', 'jhh105', 'jhh106', 'jhh107', 'jhh108', 'jhh109', 'jhh110', 'jhh111', 'jhh112', 'jhh113', 'jhh114', 'jhh115', 'jhh116', 'jhh117', 'jhh118', 'jhh119', 'jhh120', 'jhh121', 'jhh122', 'jhh124', 'jhh125', 'jhh126', 'jhh127', 'jhh128', 'jhh129', 'jhh130', 'jhh131', 'jhh132', 'jhh133', 'jhh134', 'jhh135', 'jhh136', 'jhh137', 'jhh138', 'jhh139', 'jhh140', 'jhh141', 'jhh142', 'jhh143', 'jhh144', 'jhh201', 'jhh202', 'jhh204', 'jhh205', 'jhh206', 'jhh207', 'jhh209', 'jhh210', 'jhh211', 'jhh212', 'jhh213', 'jhh214', 'jhh215', 'jhh216', 'jhh217', 'jhh218', 'j

<ipython-input-3-56c8661d2d60>:60: RuntimeWarning: invalid value encountered in true_divide
  sig = (sig - np.percentile(sig, 1))/(np.percentile(sig, 99) - np.percentile(sig, 1))
<ipython-input-3-56c8661d2d60>:60: RuntimeWarning: divide by zero encountered in true_divide
  sig = (sig - np.percentile(sig, 1))/(np.percentile(sig, 99) - np.percentile(sig, 1))


Loading data for subjects: ['jhh001', 'jhh002', 'jhh003', 'jhh004', 'jhh005', 'jhh006', 'jhh007', 'jhh008', 'jhh009', 'jhh010', 'jhh011', 'jhh012', 'jhh013', 'jhh014', 'jhh015', 'jhh016', 'jhh017', 'jhh018', 'jhh019', 'jhh020', 'jhh021', 'jhh022', 'jhh023', 'jhh024', 'jhh025', 'jhh026', 'jhh027', 'jhh028', 'jhh029', 'jhh030', 'jhh031', 'jhh032', 'jhh033', 'jhh034', 'jhh035', 'jhh036', 'jhh037', 'jhh038', 'jhh101', 'jhh102', 'jhh103', 'jhh104', 'jhh105', 'jhh106', 'jhh107', 'jhh108', 'jhh109', 'jhh110', 'jhh111', 'jhh112', 'jhh113', 'jhh114', 'jhh115', 'jhh116', 'jhh117', 'jhh118', 'jhh119', 'jhh120', 'jhh121', 'jhh122', 'jhh124', 'jhh125', 'jhh126', 'jhh127', 'jhh128', 'jhh129', 'jhh130', 'jhh131', 'jhh132', 'jhh133', 'jhh134', 'jhh135', 'jhh136', 'jhh137', 'jhh138', 'jhh139', 'jhh140', 'jhh141', 'jhh142', 'jhh143', 'jhh144', 'jhh201', 'jhh202', 'jhh204', 'jhh205', 'jhh206', 'jhh207', 'jhh209', 'jhh210', 'jhh211', 'jhh212', 'jhh213', 'jhh214', 'jhh215', 'jhh216', 'jhh217', 'jhh218', 'j

Loading data for subjects: ['jhh001', 'jhh002', 'jhh003', 'jhh004', 'jhh005', 'jhh006', 'jhh007', 'jhh008', 'jhh009', 'jhh010', 'jhh011', 'jhh012', 'jhh013', 'jhh014', 'jhh015', 'jhh016', 'jhh017', 'jhh018', 'jhh019', 'jhh020', 'jhh021', 'jhh022', 'jhh023', 'jhh024', 'jhh025', 'jhh026', 'jhh027', 'jhh028', 'jhh029', 'jhh030', 'jhh031', 'jhh032', 'jhh033', 'jhh034', 'jhh035', 'jhh036', 'jhh037', 'jhh038', 'jhh101', 'jhh102', 'jhh103', 'jhh104', 'jhh105', 'jhh106', 'jhh107', 'jhh108', 'jhh109', 'jhh110', 'jhh111', 'jhh112', 'jhh113', 'jhh114', 'jhh115', 'jhh116', 'jhh117', 'jhh118', 'jhh119', 'jhh120', 'jhh121', 'jhh122', 'jhh124', 'jhh125', 'jhh126', 'jhh127', 'jhh128', 'jhh129', 'jhh130', 'jhh131', 'jhh132', 'jhh133', 'jhh134', 'jhh135', 'jhh136', 'jhh137', 'jhh138', 'jhh139', 'jhh140', 'jhh141', 'jhh142', 'jhh143', 'jhh144', 'jhh201', 'jhh202', 'jhh204', 'jhh205', 'jhh206', 'jhh207', 'jhh209', 'jhh210', 'jhh211', 'jhh212', 'jhh213', 'jhh214', 'jhh215', 'jhh216', 'jhh217', 'jhh218', 'j

Loading data for subjects: ['jeff001', 'jeff002', 'jeff003', 'jeff004', 'jeff005', 'jeff006', 'jeff007', 'jeff008', 'jeff009', 'jeff010', 'jeff011', 'jeff012', 'jeff013', 'jeff014', 'jeff015', 'jeff016', 'jeff017', 'jeff018', 'jeff019', 'jeff020', 'jeff021', 'jeff022', 'jeff023', 'jeff024', 'jeff025', 'jeff026', 'jeff027', 'jeff028', 'jeff029', 'jeff030', 'jeff031', 'jeff032', 'jeff033', 'jeff034', 'jeff035', 'jeff036', 'jeff037', 'jeff038', 'jeff039', 'jeff040', 'jeff041', 'jeff042', 'jeff043', 'jeff044', 'jeff045', 'jeff046', 'jeff047', 'jeff048', 'jeff049', 'jeff050', 'jeff051', 'jeff052', 'jeff053', 'jeff054', 'jeff055', 'jeff056', 'jeff057', 'jeff058', 'jeff059', 'jeff060', 'jeff061', 'jeff062', 'jeff063', 'jeff064', 'jeff065', 'jeff066', 'jeff067', 'jeff101', 'jeff102', 'jeff103', 'jeff104', 'jeff105', 'jeff201', 'jeff202', 'jeff203', 'jeff204', 'jeff205', 'jeff206', 'jeff207', 'jeff208', 'jeff209', 'jeff210', 'jeff211', 'jeff212', 'jeff213', 'jeff214', 'jeff215', 'jeff216', 'jef

In [ ]:
print(thresh)

In [ ]:
# bids_path = get_bids_path_from_fname(fpath)
# edf_raw = read_raw_edf(bids_path)
# raw_data, times = raw[:, :]
# fs = edf_raw.info["sfreq"]
# T = len(edf_raw)/fs;
# thresh = -2
# minspikeLen = round(min_len * fs) # 20 ms to 70 ms is the range Anil used
# maxspikeLen = round(max_len * fs)

# # filter signal
# # nchannels * time points
# processed = np.zeros(raw_data.shape)
# spikes = np.zeros(raw_data.shape)
# for channel in np.arange(raw_data.shape[0]):
#     sig = raw_data[channel, :]
#     sig = median_filter(sig, minspikeLen)
#     sig = sig - median_filter(sig, maxspikeLen)

#     sig = (sig - np.percentile(sig, 1))/(np.percentile(sig, 99) - np.percentile(sig, 1))
#     processed[channel, :] = sig

# spikes = processed < thresh #manually chosen threshold
# print(np.sum(spikes))  

# #enforce 1 spike per region
# for chan in np.arange(spikes.shape[0]):
#     spikeSig = spikes[chan, :]
#     only = False
#     for time in np.arange(spikes.shape[1]):
#         if spikeSig[time] == 1 and only:
#             spikes[chan, time] = 0
#         elif spikeSig[time] == 0:
#             only = False
#         else:
#             only = True
# print(np.sum(spikes)) #number of unique spikes

# # KMG adding:
# # Get spike rate per channel
# ch_names = edf_raw.ch_names
# #pike_rates = np.zeros(raw_data.shape[0])
# spike_rate_dict = {}
# for chan, ch_name in enumerate(ch_names):
#     spikeLocs = np.where(spikes[chan, :] == 1)[0]
#     #pike_rates[chan] = len(spikeLocs)/T  # spikes/second
#     spike_rate_dict[ch_name] = len(spikeLocs)/T  # spikes/second
# #         print(spike_rates)

# # #Can plot spikes to see if they look okay
# # for chan in np.arange(spikes.shape[0]):
# #     chan_name = edf_raw.ch_names[chan]
# #     sig = raw_data[chan, :]
# #     spikeLocs = np.where(spikes[chan, :] == 1)[0]
# #     print(spikeLocs)
# # #     for spk in spikeLocs:
# # #         plt.figure()
# # #         plt.plot(sig[(spk-100):(spk+100)])
# # #         plt.title("Chan: " + str(chan) + " time: " + str(spk))
# # #         plt.title("Chan: "+ chan_name + " time: " + str(spk))


# # Save spike rates per channel
# deriv_root = root / "derivatives"
# # spike_root = deriv_root / "spikes_anil" / f"min-{minspikeLen}" / f"max-{maxspikeLen}" / f"thresh-{thresh}"
# spike_root = deriv_root / "spikes_anil" / f"thresh-{thresh}"
# subj_root = spike_root / f"sub-{subject}"
# subj_root.mkdir(parents=True, exist_ok=True)

# out_fpath = str(subj_root / fpath.with_suffix(".json").name) 
# with open(out_fpath, 'w+') as fid:
#     json.dump(spike_rate_dict, fid, indent=4)